# Intro

本文讲一下光栅化渲染，对比体渲染。

光栅化渲染将3D三角面投影到2D坐标下，为了模拟现实中近大远小的效果，需要先做透视投影，再计算每个射线的颜色；体渲染直接在3D空间采样射线上的多个位置来计算射线颜色，因此不需要透视投影。

两者都约定相机坐标下的相机在原点，并且相机坐标轴由$\vec{\mathbf{x}},\vec{\mathbf{y}},\vec{\mathbf{z}}$三个轴定义，其中$\vec{\mathbf{z}}$与相机面向的方向相反，$\vec{\mathbf{y}}$为相机向上的方向，$\vec{\mathbf{x}}=-\vec{\mathbf{z}}\times \vec{\mathbf{y}}$（右手定则）。



# 光栅化渲染相机变换
### 投影

<img src="../pasteImage/perspective_projection_1.png">



首先定义视锥体，渲染内容只包含视锥体内部的内容（即近平面和远平面中间的内容），如上图所示（绿色的为近平面）。

一般视锥体有关的定义有：
- 近平面左下角$(l,b,n)$和右上角$(r,t,n)$两点，一般有约定$l=-r,b=-t$。
- 长宽比$\mathrm{aspect}=r/t$
- 定义垂直方向视野角度fovy

根据以上定义（结合上图）显然有：
$$
\begin{aligned}
\tan{\frac{\mathrm{fovy}}{2}}&=\frac{t}{-n}\\
\mathrm{aspect}&=\frac{r}{t}
\end{aligned}
$$
所以给定$\mathrm{fovy},\mathrm{aspect},n$就相当于给定$l,r,b,t$。

投影变换首先希望将视锥体和射线扭曲成平行线，方便随后的正交投影，如下图（a,b）所示。透视投影前的$\mathbf{P}=(x,y,z)$在扭曲后变成了$\mathbf{P'}=(x',y',z')$，这个变化可以用一个矩阵$P'=\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}$表示：
$$
\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}=
\begin{pmatrix}
n &  &  & \\
& n & & \\
 &  & n+f & -nf\\
 &  & 1 & \\
\end{pmatrix}
$$
暂不介绍$\mathbf{M}_{\mathrm{persp}\rightarrow \mathrm{orth}}$的推导过程，因为我讲不明白。



<img src="../pasteImage/perspective_projection_2.png">



正交投影先把视锥体中心平移到原点，再放缩进canonical cube里，最后取消z轴即可。因为整个过程只涉及到平移和放缩，所以$\mathbf{M}_\mathrm{orth}$如下：
$$
\mathbf{M}_\mathrm{orth}=
\begin{pmatrix}
\frac{2}{r-l} &  &  & \\
& \frac{2}{t-b} & & \\
 &  & \frac{2}{n-f} & \\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
1 &  &  & -\frac{r+l}{2}\\
& 1 & & -\frac{t+b}{2}\\
 &  & 1 & -\frac{n+f}{2}\\
 &  & 1 & \\
\end{pmatrix}
$$

整个投影过程为$\mathbf{M}_{\mathrm{persp}\rightarrow\mathrm{orth}}\mathbf{M}_\mathrm{orth}$：
$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
\frac{2}{r-l} &  &  & \\
& \frac{2}{t-b} & & \\
 &  & \frac{2}{n-f} & \\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
1 &  &  & -\frac{r+l}{2}\\
& 1 & & -\frac{t+b}{2}\\
 &  & 1 & -\frac{n+f}{2}\\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
n &  &  & \\
& n & & \\
 &  & n+f & -nf\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
\frac{n}{r} &  &  & \\
& \frac{n}{t} & & \\
 &  & \frac{n+f}{n-f} & \frac{-2nf}{n-f}\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
\frac{1}{\tan{\frac{\mathrm{fovy}}{2}}*\mathrm{aspect}} &  &  & \\
& \frac{1}{\tan{\frac{\mathrm{fovy}}{2}}} & & \\
 &  & \frac{n+f}{n-f} & \frac{-2nf}{n-f}\\
 &  & 1 & \\
\end{pmatrix}
$$

注意到上式和OpenGL的glFrustum里的矩阵不一样，因为此矩阵里的$u,f$为距离值（正值）而非坐标值；而上式子里的$u,f$为带符号坐标值（负值）。我们不妨令$f'=-f,n'=-n$，并且乘一个翻转z轴的缩放矩阵。

$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
1 &  &  & \\
& 1 & & \\
 &  & -1 &\\
 &  &  & 1\\
\end{pmatrix}
\begin{pmatrix}
-\frac{n'}{r} &  &  & \\
& -\frac{n'}{t} & & \\
 &  & -\frac{n'+f'}{f'-n'} & -\frac{2n'f'}{f'-n'}\\
 &  & 1 & \\
\end{pmatrix}
=
\begin{pmatrix}
-\frac{n'}{r} &  &  & \\
& -\frac{n'}{t} & & \\
 &  & \frac{n'+f'}{f'-n'} & \frac{2n'f'}{f'-n'}\\
 &  & 1 & \\
\end{pmatrix}
$$
不妨整体乘以-1，因为齐次坐标同时乘以一个系数和原来的坐标是一样的，于是我们得到和OpenGL相同的投影矩阵：
$$
\mathbf{M}_\mathrm{persp}=
\begin{pmatrix}
\frac{n'}{r} &  &  & \\
& \frac{n'}{t} & & \\
 &  & -\frac{n'+f'}{f'-n'} & -\frac{2n'f'}{f'-n'}\\
 &  & -1 & \\
\end{pmatrix}
$$

### 旋转
旋转过程的目的是将世界坐标转化为相机坐标，$\mathbf{R}_{w2c}$的推导详见3D/viewport_tranformation.ipynb。最终的投影结果为：$\mathbf{M}_\mathrm{persp}\mathbf{R}_{w2c}$。